# Домашнее задание по курсу анализа данных # 2
Версия от 14.11.2021

Всего 50 баллов = 50 основных + 10 дополнительных

**Оценка**  = (Количество баллов за работу*2 + 20)/10

**Мягкий дедлайн: 06.12.2021  23:59**

**Жесткий дедлайн: 11.12.2021  23:59**

Присылать решение ссылкой на ваш **github** в телеграмм **@Aleron75** или на почту **milenkin.aa@phystech.edu**

**Автор:** Миленькин Александр

# Парсим рейтинг преподавателей с Вики-Физтех

<div class="alert alert-info">
    
Итак, перед вами **VIKI-MIPT** - сборник отзывов о кафедрах и преподавателях МФТИ: ` http://wikimipt.org/wiki/ `

Эта кладезь отзывов не раз давала студентам оценить психологический портрет потенциальных экзаменаторов и преподавателей. 
    
    
<img src="../lessons/images/wiki_mipt_front.PNG" height="300" width="1300"> 



# Описание задачи: 

**Задача** - собрать данные по всем текущим преподавателям всех `институтских кафедр` МФТИ.

Собранный датасет должен представлять из себя `.csv` файл со следующими полями:

* `full_name` - ФИО преподавателя
*  `teach_place` - Кафедра
*  `degree`- Ученая Степень
* `knowledge` - Знания
*  `teaching_skill` - Умение преподавать	
* `commication_skill` - В общении	
* `easy_exam` - «Халявность»	
* `overall_score` - Общая оценка

<img src="../lessons/images/wiki_mipt_info.PNG" height="300" width="300"> 

**Комментарий:** - Знание, умение преподавать, в общение, "Халявность" и Общая оценка можно найти странице каждого преподавателя. 

<div class="alert alert-warning">
    
Финальный балл складывается из трех частей:

* Cобранный датасет (выложить архивом на гитхаб)
* python кода, которым был собран датасет (выложить на гитхаб)
* Окуратный и понятный код с комментариями
    
    
**Дополнительные + 10 баллов можно получить за кластеризацию преподавателей, используя только оценки из собранных данных**

# Алгоритм обхода для сбора данных:

### 0 - Открываем с помощью Selenium главную страницу вики-Физтех- http://wikimipt.org/wiki/   
### 1 - Получаем список кафедр с главной страницу вики-Физтех    
Получаем страницу кафедры как http://wikimipt.org/wiki/<название кафедры>
### 2 - Получаем список преподавателей со страницы кафедры
Получаем страницу преподавателя как http://wikimipt.org/wiki/<фио преподавателя>
### 3 - Со страницы преподавателя получаем табличку данных с характеристмиками преподавателями и прочую необходимую информацию. 
### 4 - Записываем в pandas dataframe и сохраняем данные в .csv файл

Самые большие проблемы, которые были:
1) Таблица с нужными данными очень часто не прогружается на сайте, даже если ждать какое-то время, причем не загружаются именно оценки
(пофиксил с помощью WebDriverWait и обновлением страницы в случае, если он не помог)

2) У некоторых кафедр таблицы с препами находятся в другом месте, а у кафедры биофизики и экологии страница вообще не сделана, хотя ссылка на нее есть на главной
(убрал эту кафедру, а остальным особенным прописал свои пути)

3) ~~поленился~~ не успел оптимизировать код, чтобы он работал побыстрее, данные парсились около получаса

In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import quote

Инициируем массивы, где будем хранить данные о препах и прописываем url и параметры хрома

In [2]:
names = np.empty([])
teach_places = np.empty([], dtype="float16")
knowledges = np.empty([], dtype="float16")
teaching_skills = np.empty([], dtype="float16")
communication_skills = np.empty([], dtype="float16")
easy_exams = np.empty([], dtype="float16")
overall_scores = np.empty([], dtype="float16")
degrees = np.empty([])

s = Service('./chromedriver')
url = "http://wikimipt.org/wiki/"
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--headless")

Чтобы оценки были численные, а у тех, у кого нет оценки были nan

In [3]:
def to_float(string):
    try:
        a = float(string)
    except ValueError:
        a = np.nan
    return a

## Основной суп

In [4]:
driver = webdriver.Chrome(service=s, options=chrome_options)
driver.get(url)

kafs = driver.find_elements(By.XPATH,'//*[@id="mw-content-text"]/div[3]') # ищем таблицу с кафедрами
kafs = [i.text for i in kafs]
kafs = kafs[0].split('\n')
kafs.remove('Кафедра биофизики и экологии') # Убираем кафедру, страница которой не доделана
special = {"Кафедра системного анализа экономики" : '//*[@id="mw-content-text"]/div[5]', "Военная кафедра" : '//*[@id="61ad261203d24"]', \
    "Кафедра философии": '//*[@id="61ad261b1f678"]', "Высшая школа системного инжиниринга" : '//*[@id="61ad262e04944"]'}
    # Прописал отдельно пути для кафедр, у который список преподавателей находится в другом месте
for kaf in kafs: 
    path = "_".join([quote(i) for i in kaf.split()]) # Создаю url страницы кафедры
    driver.get(url+ path)
    if kaf in special.keys():
        preps = driver.find_elements(By.XPATH, special[kaf])
    else:
        preps = driver.find_elements(By.XPATH,'//*[@id="mw-content-text"]/div[3]')
    preps = [i.text for i in preps]
    preps = preps[0].split('\n')
    for prep in preps:
        prep_path = "_".join([quote(i) for i in prep.split()]) # создаю url страницы преподавателя
        driver.get(url+prep_path)

        # Танцы с бубном для того, чтобы нужная таблица полностью прогрузилась
        try:
            # Ждём либо 20 секунд, либо пока не прогрузятся все элементы с тэгом 'tr'
            wait = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
            info =  driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/table') # забираем таблицу
            table = pd.read_html(info[0].get_attribute('outerHTML'))[0] # грузим в Pandas
            know = table[table[0]=="Знания"][1].values[0].split()[0].removeprefix('01234') # Пытаемся обратиться к оценке
        except:
            driver.refresh() # Обновляем страницу в случае, если значения оценок не прогрузились в таблице или
                             # сама страница не прогрузилась за 20 с
            wait = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
        
        info =  driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/table') #тоже самое, что в try
        table = pd.read_html(info[0].get_attribute('outerHTML'))[0]

        # Забираем нужные данные
        deg = table[table[0]=="Учёная степень"][1].values
        know = table[table[0]=="Знания"][1].values[0].split()[0].removeprefix('01234')
        teach_s = table[table[0]=="Умение преподавать"][1].values[0].split()[0].removeprefix('01234')
        comun_s = table[table[0]=="В общении"][1].values[0].split()[0].removeprefix('01234')
        easy_e = table[table[0]=="«Халявность»"][1].values[0].split()[0].removeprefix('01234')
        overall_s = table[table[0]=="Общая оценка"][1].values[0].split()[0].removeprefix('01234')

        # Сохраняем
        names = np.append(names, prep)
        teach_places = np.append(teach_places, kaf)
        knowledges = np.append(knowledges, to_float(know))
        teaching_skills = np.append(teaching_skills, to_float(teach_s))
        communication_skills = np.append(communication_skills, to_float(comun_s))
        easy_exams = np.append(easy_exams, to_float(easy_e))
        overall_scores = np.append(overall_scores, to_float(overall_s))
        if len(deg) == 0:
            degrees = np.append(degrees, np.nan)
        else:
            degrees = np.append(degrees, deg)

# Finally, создаем DataFrame
wiki_mipt = pd.DataFrame({'full_name': names, 'teach_place': teach_places, 'degree': degrees, 'knowledge': knowledges, \
    'teaching_skill': teaching_skills, 'commication_skill': communication_skills, 'easy_exam': easy_exams, 'overall_score': overall_scores})

Последний штрих, выкидываем 0 строчку, так как там лежат значения для инициализации массивов

In [9]:
wiki_mipt.drop(0, inplace=True)

Та -да!

In [10]:
wiki_mipt

,full_name,teach_place,degree,knowledge,teaching_skill,commication_skill,easy_exam,overall_score
1,Агаханов Назар Хангельдыевич,Кафедра высшей математики,Кандидат физико-математических наук (1988),4.86,4.77,4.79,4.25,4.79
2,Агаханова Яна Сергеевна,Кафедра высшей математики,Кандидат физико-математических наук (2007),4.51,4.63,4.85,4.81,4.81
3,Алексеев Василий Антонович,Кафедра высшей математики,NaN,4.67,4.33,4.50,4.17,4.67
4,Амосов Григорий Геннадьевич,Кафедра высшей математики,Доктор физико-математических наук,4.42,3.42,4.33,4.44,4.00
5,Андреев Михаил Юрьевич,Кафедра высшей математики,кандидат физико-математических наук,3.46,3.50,3.73,3.47,3.69
...,...,...,...,...,...,...,...,...
742,Лупандин Иван Владимирович,Кафедра философии,NaN,4.67,4.50,4.67,4.50,4.67
743,Петров Алексей Алексеевич,Кафедра философии,NaN,NaN,NaN,NaN,NaN,NaN
744,Семёнов Юрий Иванович,Кафедра философии,Доктор исторических наук,NaN,NaN,NaN,NaN,NaN
745,Скворчевский Константин Анатольевич,Кафедра философии,Доктор технических наук,4.43,4.43,4.29,3.50,4.33


In [11]:
wiki_mipt.to_csv("wikimipt.csv")

Код, к сожалению, работает около получаса, поэтому ниже приложу уже готовый датасет

In [16]:
Wiki_mipt = pd.read_csv("./data/wikimipt.csv")
wiki_mipt

,full_name,teach_place,degree,knowledge,teaching_skill,commication_skill,easy_exam,overall_score
1,Агаханов Назар Хангельдыевич,Кафедра высшей математики,Кандидат физико-математических наук (1988),4.86,4.77,4.79,4.25,4.79
2,Агаханова Яна Сергеевна,Кафедра высшей математики,Кандидат физико-математических наук (2007),4.51,4.63,4.85,4.81,4.81
3,Алексеев Василий Антонович,Кафедра высшей математики,NaN,4.67,4.33,4.50,4.17,4.67
4,Амосов Григорий Геннадьевич,Кафедра высшей математики,Доктор физико-математических наук,4.42,3.42,4.33,4.44,4.00
5,Андреев Михаил Юрьевич,Кафедра высшей математики,кандидат физико-математических наук,3.46,3.50,3.73,3.47,3.69
...,...,...,...,...,...,...,...,...
742,Лупандин Иван Владимирович,Кафедра философии,NaN,4.67,4.50,4.67,4.50,4.67
743,Петров Алексей Алексеевич,Кафедра философии,NaN,NaN,NaN,NaN,NaN,NaN
744,Семёнов Юрий Иванович,Кафедра философии,Доктор исторических наук,NaN,NaN,NaN,NaN,NaN
745,Скворчевский Константин Анатольевич,Кафедра философии,Доктор технических наук,4.43,4.43,4.29,3.50,4.33


# Вспомогательные элементы для майнинга данных

* Библиотека `Selenium`: https://selenium-python.readthedocs.io/

Полезные методы и рекомендации:
    
* `element = browser.find_elements_by_*что угодно*()` # Поиск элемента на странице
* `element.click()` # Кликнуть по элементу (провалится по ссылке / расскрыть банер)
* `browser.back()` # Вернуться назад 
* `pd.read_html()` # Способ загрузить таблицку из html сразу в Pandas
* Спрашивать в чате или гуглить